# Wess Zumino 

## N=1 in 1+1

The Hamiltonian with a central difference for the derivative is given by 

\begin{align}
    H=\sum_n&\left[ \frac{p_n^2}{2a} + \frac{a}{2}\left(\frac{\phi_{n+1}-\phi_{n-1}}{2a}\right)^2+\frac{a}{2}V(\phi_n)^2
            +aV(\phi_n)\frac{\phi_{n+1}-\phi_{n-1}}{2a} \right. \nonumber \\
            &\left.+(-1)^nV'(\phi_n)\left(\chi_n^{\dagger}\chi_n-\frac{1}{2}\right)
            +\frac{1}{2a}\left(\chi_n^{\dagger}\chi_{n+1}+\chi_{n+1}^{\dagger}\chi_n\right) \right],
\end{align}

In [1]:
import sympy as sp
from ham_to_sparse_matrix import *
from constants import *


In [2]:
bosonNI=sp.expand( pn**2/(2*aLat) + (aLat/2)*((qnP1-qnM1)/(2*aLat))**2 )
bosonI=sp.expand( (aLat/2)*V(qn)**2 + aLat*V(qn)*(qnP1-qnM1)/(2*aLat) )
fermionNI=sp.expand( (xdnP1*xn+xdn*xnP1)/(2*aLat) )
fermionI=sp.expand( sp.diff(V(qn),qn)*(xdn*xn-(1/2)) )

In [3]:
bosonNI+bosonI

a*V(q_{n})**2/2 + V(q_{n})*q_{n+1}/2 - V(q_{n})*q_{n-1}/2 + p_{n}**2/(2*a) - q_{n+1}*q_{n-1}/(8*a) + q_{n+1}**2/(8*a) - q_{n-1}*q_{n+1}/(8*a) + q_{n-1}**2/(8*a)

In [4]:
fermionNI + fermionI

-0.5*Derivative(V(q_{n}), q_{n}) + Derivative(V(q_{n}), q_{n})*\chi^{\dagger}_{n}*\chi_{n} + \chi^{\dagger}_{n+1}*\chi_{n}/(2*a) + \chi^{\dagger}_{n}*\chi_{n+1}/(2*a)

In [4]:
N=3
aVal=1

# depends on finite-difference method
qs=[SiteSymbol('q',str(i)) for i in range(-1,N+1,1)]
# maybe make qs a normal site list
# and make an extra boundaryQs list for q[-1], q[N]
# counting would be normal computer science way for rest of code.

ps=[SiteSymbol('p',str(i)) for i in range(-1,N+1,1)] # don't really need extras
aops=[SiteSymbol('a',str(i)) for i in range(-1,N+1,1)]
adags=[SiteSymbol('a^{\dagger}',str(i)) for i in range(-1,N+1,1)]
xs=[SiteSymbol('\chi',str(i)) for i in range(-1,N+1,1)]
xdags=[SiteSymbol('\chi^{\dagger}',str(i)) for i in range(-1,N+1,1)]

# note this is exactly hardcoded for this finite difference method.
bcType = 'periodic'
boundaryConditions = {}
if bcType == 'periodic':
    boundaryConditions = {qs[0]: qs[N], qs[N+1]: qs[1],
                          xs[0]: -xs[N], xs[N+1]: -xs[1], 
                          xdags[0]: -xdags[N],  xdags[N+1]: -xdags[1]
                         }
    
elif bcType == 'dirichlet':
    boundaryConditions = { qs[0]: 0, qs[N+1]: 0,
                          xs[0]: 0, xs[N+1]: 0,
                          xdags[0]: 0, xdags[N+1]:0
                         }

In [5]:
totHam=bosonNI+bosonI+fermionNI+fermionI
totHam

a*V(q_{n})**2/2 + V(q_{n})*q_{n+1}/2 - V(q_{n})*q_{n-1}/2 - 0.5*Derivative(V(q_{n}), q_{n}) + Derivative(V(q_{n}), q_{n})*\chi^{\dagger}_{n}*\chi_{n} + \chi^{\dagger}_{n+1}*\chi_{n}/(2*a) + \chi^{\dagger}_{n}*\chi_{n+1}/(2*a) + p_{n}**2/(2*a) - q_{n+1}*q_{n-1}/(8*a) + q_{n+1}**2/(8*a) - q_{n-1}*q_{n+1}/(8*a) + q_{n-1}**2/(8*a)

In [7]:
ham=0

mass=1

for i in range(1,N+1):
    ham+=(bosonNI+bosonI+fermionNI).subs({
        pn: ps[i],
        qn: qs[i], qnP1: qs[i+1], qnM1: qs[i-1],
        xn: xs[i], xnP1: xs[i+1],
        xdn: xdags[i], xdnP1: xdags[i+1]
    }).subs(boundaryConditions)
    
# ham.subs(boundaryConditions).doit() # this doesn't work?


def potential(n):
    # m*q with m=1
    return mass*qs[n]

potentialSubs={}
for n in range(1,N+1):
    potentialSubs[V(qs[n])]=potential(n)

ham=sp.simplify(ham.subs(potentialSubs).subs(aLat,aVal))

for i in range(1,N+1):
    if i%2==0:
        ham+=sp.simplify(fermionI.subs({qn: qs[i], xn: xs[i], xdn: xdags[i]}).subs(potentialSubs))
    else:
        ham-=sp.simplify(fermionI.subs({qn: qs[i], xn: xs[i], xdn: xdags[i]}).subs(potentialSubs))
ham=sp.simplify(ham.subs(aLat,aVal))
ham

(4.0 - 8*\chi^{\dagger}_{0}*\chi_{0} + 4*\chi^{\dagger}_{0}*\chi_{1} - 4*\chi^{\dagger}_{0}*\chi_{2} + 4*\chi^{\dagger}_{1}*\chi_{0} + 8*\chi^{\dagger}_{1}*\chi_{1} + 4*\chi^{\dagger}_{1}*\chi_{2} - 4*\chi^{\dagger}_{2}*\chi_{0} + 4*\chi^{\dagger}_{2}*\chi_{1} - 8*\chi^{\dagger}_{2}*\chi_{2} + 4*p_{0}**2 + 4*p_{1}**2 + 4*p_{2}**2 + 3*q_{0}*q_{1} - 5*q_{0}*q_{2} + 6*q_{0}**2 - 5*q_{1}*q_{0} + 3*q_{1}*q_{2} + 6*q_{1}**2 + 3*q_{2}*q_{0} - 5*q_{2}*q_{1} + 6*q_{2}**2)/8

In [9]:
m=1

HOdofSubs = {}
#offset because of BC
for i in range(1,N+1):
    HOdofSubs[qs[i]] = 0.5*sp.sqrt(2/m)*(aops[i] + adags[i])
    HOdofSubs[ps[i]] = complex(0,1)*sp.sqrt(2*m)*(adags[i] - aops[i])/2 

hoHam=sp.expand(ham.subs(HOdofSubs))
hoHam=sp.nsimplify(hoHam,tolerance=1e-8)
hoHam

1/2 - \chi^{\dagger}_{0}*\chi_{0} + \chi^{\dagger}_{0}*\chi_{1}/2 - \chi^{\dagger}_{0}*\chi_{2}/2 + \chi^{\dagger}_{1}*\chi_{0}/2 + \chi^{\dagger}_{1}*\chi_{1} + \chi^{\dagger}_{1}*\chi_{2}/2 - \chi^{\dagger}_{2}*\chi_{0}/2 + \chi^{\dagger}_{2}*\chi_{1}/2 - \chi^{\dagger}_{2}*\chi_{2} + 3*a^{\dagger}_{0}*a^{\dagger}_{1}/16 - 5*a^{\dagger}_{0}*a^{\dagger}_{2}/16 + 5*a^{\dagger}_{0}*a_{0}/8 + 3*a^{\dagger}_{0}*a_{1}/16 - 5*a^{\dagger}_{0}*a_{2}/16 + a^{\dagger}_{0}**2/8 - 5*a^{\dagger}_{1}*a^{\dagger}_{0}/16 + 3*a^{\dagger}_{1}*a^{\dagger}_{2}/16 - 5*a^{\dagger}_{1}*a_{0}/16 + 5*a^{\dagger}_{1}*a_{1}/8 + 3*a^{\dagger}_{1}*a_{2}/16 + a^{\dagger}_{1}**2/8 + 3*a^{\dagger}_{2}*a^{\dagger}_{0}/16 - 5*a^{\dagger}_{2}*a^{\dagger}_{1}/16 + 3*a^{\dagger}_{2}*a_{0}/16 - 5*a^{\dagger}_{2}*a_{1}/16 + 5*a^{\dagger}_{2}*a_{2}/8 + a^{\dagger}_{2}**2/8 + 5*a_{0}*a^{\dagger}_{0}/8 + 3*a_{0}*a^{\dagger}_{1}/16 - 5*a_{0}*a^{\dagger}_{2}/16 + 3*a_{0}*a_{1}/16 - 5*a_{0}*a_{2}/16 + a_{0}**2/8 - 5*a_{1}*a^{\da

In [14]:
cutoff=6
hamMat=convert_to_matrix(hoHam,cutoff,N,aops,adags,xs,xdags)

In [15]:
import scipy.sparse.linalg

In [18]:
hamMat

<1728x1728 sparse matrix of type '<class 'numpy.complex64'>'
	with 25632 stored elements in Compressed Sparse Row format>

In [25]:
for cutoff in [2,3,4,5,6,7,8,9,10,11,12]:
    t=Timer('toHam')
    t.start()
    hamMat=convert_to_matrix(hoHam,cutoff,N,aops,adags,xs,xdags)
    t.stop()
    
    t=Timer('evals')
    t.start()
    ens=scipy.sparse.linalg.eigs(hamMat,k=10,sigma=0)[0]
    t.stop()
    
    print("Lambda = ",cutoff)
    print(ens)
    print()

toHam timer took: 0.3371 seconds
evals timer took: 0.0019 seconds
Lambda =  2
[-0.28077638-7.2391173e-09j  0.21922363+5.6406044e-09j
 -0.2807765 -1.1316464e-08j  0.21922354+8.1443012e-09j
  0.31385946-8.8072980e-09j  0.31385922-8.3646805e-09j
  0.2192235 -3.5939138e-10j  0.21922342-3.5806677e-10j
  0.21922368+3.1709821e-09j  0.2192235 -1.8226245e-09j]

toHam timer took: 0.5952 seconds
evals timer took: 0.0036 seconds
Lambda =  3
[-0.04814541+4.5906208e-09j  0.5464892 -7.9585121e-09j
  0.6309455 +3.0804053e-08j  0.6630059 -3.0256707e-08j
  0.663006  -4.3429349e-09j  1.0731779 +1.2821408e-08j
  1.2192225 -1.7008501e-08j  1.2255825 -1.5325528e-08j
  1.2326298 -7.0213351e-09j  1.252933  -3.2360180e-08j]

toHam timer took: 1.0964 seconds
evals timer took: 0.0063 seconds
Lambda =  4
[0.04155127+2.9033738e-09j 0.6361867 +9.8910879e-09j
 0.8569632 +8.2918907e-09j 1.0701864 +5.2265488e-08j
 1.070188  +3.1466058e-08j 1.3223246 +5.5100813e-09j
 1.4515995 +2.0447617e-08j 1.4790865 -5.0473894e-08j
